In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
import math

In [2]:
data = pd.read_csv("Stock return.csv", encoding = "ANSI")
Fama = pd.read_csv("Fama.monthly.data.txt", sep="\t")
three_factor = Fama.loc[Fama["Date"] >= 200001,]
stock_price = pd.read_csv("Stock price.csv", encoding = "ANSI")

In [3]:
date = []
for i in range(18):
    date.append(2000 + i)
col = []
for j in data.columns[1:]:
    col.append(j)
IVOL = pd.DataFrame(index = date, columns = col)

In [4]:
for r in range(18):
    for c in range(data.shape[1] - 1):
        a = r * 12
        b = 12 * (r + 1)
        if data.iloc[a:b,c + 1].isna().sum() > 6:
            IVOL.iloc[r,c] = np.nan
        else:
            x = three_factor.iloc[a:b,1:4]
            y = data.iloc[a:b,c + 1]
            x = x[list(y.isna()==False)]
            y = y[y.isna()==False]
            Reg = LinearRegression().fit(x,y)
            IVOL.iloc[r,c] = np.std(y - Reg.predict(x), ddof = 1)

In [5]:
stock_price = stock_price.fillna(0)
Return = pd.DataFrame(index = date, columns = col)
for x in range(18):
    for y in range(Return.shape[1]):
        if stock_price.iloc[x,y + 1]==0 or stock_price.iloc[x + 1,y + 1]==0:
            Return.iloc[x,y] = np.nan
        else:
            Return.iloc[x,y] = (stock_price.iloc[x + 1,y + 1] - stock_price.iloc[x,y + 1]) / stock_price.iloc[x,y + 1]

In [6]:
t = ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8", "G9", "G10"]
Group = pd.DataFrame(index = date, columns = t)

In [7]:
def outcome(p):
    IVOL_p = pd.DataFrame(IVOL.iloc[p])
    Return_p = pd.DataFrame(Return.iloc[p])
    outcome_p = pd.concat([IVOL_p, Return_p], axis = 1, join = "inner")
    outcome_p.columns = ["IVOL", "Return"]
    outcome_p = outcome_p.sort_values(by = "IVOL", ascending=False)
    return outcome_p

In [8]:
for q in range(18):
    outcome_grouping = outcome(q)
    outcome_grouping = outcome_grouping.dropna(axis = 0, how = "any")
    w = math.floor(outcome_grouping.shape[0] / 10)
    v = 9 * w
    u = 10 * w
    grouping = np.array(outcome_grouping.iloc[:u,1]).reshape(10,w)
    for z in range(10):
        if z==9:
            Group.iloc[q,9] = str(round(np.mean(outcome_grouping.iloc[v:,1]) * 100, 2)) + "%"
        else:
            Group.iloc[q,z] = str(round(np.mean(grouping[z]) * 100, 2)) + "%"

In [9]:
Group

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10
2000,-36.89%,-37.35%,-49.06%,-40.17%,-40.85%,-42.54%,-40.33%,-39.08%,-41.16%,-36.97%
2001,43.79%,27.13%,16.69%,22.57%,7.92%,18.5%,-3.49%,2.63%,-3.77%,-2.8%
2002,100.84%,82.59%,35.95%,21.35%,22.11%,13.16%,-14.02%,0.55%,-4.81%,2.69%
2003,79.72%,54.53%,35.47%,31.87%,22.9%,22.22%,21.07%,18.43%,15.55%,10.39%
2004,-7.31%,-11.91%,-22.27%,-19.83%,-19.1%,-15.59%,-17.6%,-8.99%,-14.36%,-7.15%
2005,74.36%,39.18%,24.68%,5.93%,11.27%,6.77%,9.73%,-0.33%,0.16%,-4.06%
2006,130.05%,69.4%,60.7%,44.06%,48.29%,38.31%,33.1%,33.19%,26.88%,18.13%
2007,13.18%,-1.73%,-5.63%,-3.44%,-6.9%,-10.29%,-12.52%,-10.19%,-6.45%,-7.62%
2008,-27.44%,-41.19%,-46.37%,-42.39%,-41.43%,-41.54%,-41.78%,-38.12%,-38.72%,-36.75%
2009,242.59%,198.65%,183.24%,137.8%,141.39%,133.03%,116.17%,96.26%,82.85%,68.24%
